In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
### 데이터 처리


# 데이터 읽기
import pandas as pd

read_train=pd.read_csv('/content/gdrive/My Drive/train.csv')
read_test = pd.read_csv('/content/gdrive/My Drive/test.csv')




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  # Remove the CWD from sys.path while we load stuff.


In [256]:
data_train=read_train #[["Survived","Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]] # data categories
data_test = read_test #[["PassengerId","Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]]

### 결측값 채워넣기
import numpy as np
# 대체되는 값에 따라서 성능이 좌지우지될 수 있기에 신중하게!!

# 이름으로부터 성별 추측
data_train['Initial']= data_train.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations
data_test['Initial']= data_test.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations

# 결측값이 있는 데이터에 대해서 임의적으로 값을 집어넣어주어야 함
#data_train=data_train.dropna(how="any") # dropna: 결측값(NaN)있는 행을 제거, how("any"): 결측값이 하나라도 있을 경우 drop, how("all"): 모든 값이 결측값일 경우 drop 



### 결측값 채워넣기
import numpy as np
# 대체되는 값에 따라서 성능이 좌지우지될 수 있기에 신중하게!!

# 이름으로부터 성별 추측
data_train['Initial']= read_train.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations
data_test['Initial']= read_test.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations

#print(data_train.head())
#print(data_train.head())
#pd.crosstab(data_train['Initial'], data_train['Sex']).T.style.background_gradient(cmap='summer_r') #Checking the Initials with the Sex 

# 성별을 알맞게 바꿔넣기
data_train['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)

data_test['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)



#print('train 데이터에의 성별에 값에 대한 평균')
#print(data_train.groupby('Initial').mean()) # 남성보다는 여성의 생존율이 더 높음을 볼 수 있음
# 그룹별
grouped_train = data_train.groupby(['Sex','Pclass', 'Age'])
data_train.Age = grouped_train.Age.apply(lambda x: x.fillna(x.median()))

grouped_test = data_test.groupby(['Sex','Pclass', 'Age'])
data_test.Age = grouped_test.Age.apply(lambda x: x.fillna(x.median()))

# # 나이 채워넣기
# data_train.loc[(data_train.Age.isnull())&(data_train.Initial=='Mr'),'Age'] = 33
# data_train.loc[(data_train.Age.isnull())&(data_train.Initial=='Mrs'),'Age'] = 36
# data_train.loc[(data_train.Age.isnull())&(data_train.Initial=='Master'),'Age'] = 5
# data_train.loc[(data_train.Age.isnull())&(data_train.Initial=='Miss'),'Age'] = 22
# data_train.loc[(data_train.Age.isnull())&(data_train.Initial=='Other'),'Age'] = 46

# data_test.loc[(data_test.Age.isnull())&(data_test.Initial=='Mr'),'Age'] = 33
# data_test.loc[(data_test.Age.isnull())&(data_test.Initial=='Mrs'),'Age'] = 36
# data_test.loc[(data_test.Age.isnull())&(data_test.Initial=='Master'),'Age'] = 5
# data_test.loc[(data_test.Age.isnull())&(data_test.Initial=='Miss'),'Age'] = 22
# data_test.loc[(data_test.Age.isnull())&(data_test.Initial=='Other'),'Age'] = 46
def category_age(x):
    if x <15:
        return 0
    elif x < 25:
        return 1
    elif x < 35:
        return 2
    elif x < 45:
        return 3
    elif x < 55:
        return 4
    elif x < 65:
        return 5
    elif x < 75:
        return 6
    else:
        return 7    
data_train['Age_cate'] = data_train['Age'].apply(category_age)
data_test['Age_cate'] = data_test['Age'].apply(category_age)

# 성별
data_train['Initial'] = data_train['Initial'].replace({'Master': 0, 'Miss': 1, 'Mr': 2, 'Mrs': 3, 'Other': 4})
data_test['Initial'] = data_test['Initial'].replace({'Master': 0, 'Miss': 1, 'Mr': 2, 'Mrs': 3, 'Other': 4})

# Embarked 채워 넣기
data_train['Embarked'].fillna('S', inplace=True)

# Fare 채워 넣기
data_test.loc[data_test.Fare.isnull(), 'Fare'] = data_test['Fare'].median()
def category_fare(x):
  if x<=30:
    return 0
  elif x<=80:
    return 1
  elif x<=100:
    return 2
  else:
    return 3
data_train['Fare'] = data_train['Fare'].apply(category_fare)
data_test['Fare'] = data_test['Fare'].apply(category_fare)

# 문자형식으로 저장되어 있는 성(sex)과 탑승역(embarked)에 대해 숫자로 치환
data_train=data_train.replace({"male":0,"female":1,"Q":0,"S":1,"C":2}) 
data_test=data_test.replace({"male":0,"female":1,"Q":0,"S":1,"C":2})

print(data_train.head())
#print(data_test.isnull().any()) 


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    0  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1      0   
2                             Heikkinen, Miss. Laina    1  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1      0   
4                           Allen, Mr. William Henry    0  35.0      0      0   

             Ticket  Fare Cabin  Embarked  Initial  Age_cate  
0         A/5 21171     0   NaN         1        2         1  
1          PC 17599     1   C85         2        3         3  
2  STON/O2. 3101282     0   NaN         1        1         2  
3            113803     1  C123         1   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but

In [262]:
# 정규화 작업
from sklearn.preprocessing import StandardScaler
data_train['Pclass'] = StandardScaler().fit_transform(data_train['Pclass'].values.reshape(-1, 1))
data_train['Sex'] = StandardScaler().fit_transform(data_train['Sex'].values.reshape(-1, 1))
data_train['Age'] = StandardScaler().fit_transform(data_train['Age'].values.reshape(-1, 1))
data_train['Fare'] = StandardScaler().fit_transform(data_train['Fare'].values.reshape(-1, 1))
data_train['Embarked'] = StandardScaler().fit_transform(data_train['Embarked'].values.reshape(-1, 1))
data_train['Initial'] = StandardScaler().fit_transform(data_train['Initial'].values.reshape(-1, 1))
data_train['Age_cate'] = StandardScaler().fit_transform(data_train['Age_cate'].values.reshape(-1, 1))

data_test['Pclass'] = StandardScaler().fit_transform(data_test['Pclass'].values.reshape(-1, 1))
data_test['Sex'] = StandardScaler().fit_transform(data_test['Sex'].values.reshape(-1, 1))
data_test['Age'] = StandardScaler().fit_transform(data_test['Age'].values.reshape(-1, 1))
data_test['Fare'] = StandardScaler().fit_transform(data_test['Fare'].values.reshape(-1, 1))
data_test['Embarked'] = StandardScaler().fit_transform(data_test['Embarked'].values.reshape(-1, 1))
data_test['Initial'] = StandardScaler().fit_transform(data_test['Initial'].values.reshape(-1, 1))
data_test['Age_cate'] = StandardScaler().fit_transform(data_test['Age_cate'].values.reshape(-1, 1))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by Stand

In [272]:
# 트레이닝할 데이터의 입력 및 출력 데이터 나눔
X_train=data_train[["Pclass", "Initial", "Age_cate", "Fare","Embarked"]].as_matrix()
Y_train=data_train[["Survived"]].as_matrix()
# 입력, 히든, 출력층의 길이 지정
len_in=len(X_train[0])
len_out= 1#len(Y_train)
n_hidden = 100

print('in', len_in)
# 테스트할 데이터
p_id=data_test[["PassengerId"]].as_matrix()
X_test = data_test[["Pclass", "Initial", "Age_cate","Fare","Embarked"]].as_matrix()

from sklearn.model_selection import train_test_split
rf_X_train = X_train
rf_Y_train = Y_train
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.2, random_state=4242)

in 5


In [0]:
# 모델 생성
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Activation ,Dropout, merge, add
from keras.optimizers import SGD, Adam, Adagrad, Adadelta
from keras.layers.normalization import BatchNormalization

def build_model():
  
  model = Sequential()
  model .add(Dense(n_hidden, input_dim=len_in))
  model.add(BatchNormalization())
  model .add(Activation('relu'))
  #model.add(Dropout(0.3))
  
  model .add(Dense(n_hidden))
  model.add(BatchNormalization())
  model .add(Activation('relu'))
  #model.add(Dropout(0.3))

  model .add(Dense(n_hidden))
  model.add(BatchNormalization())
  model .add(Activation('relu'))
  model.add(Dropout(0.3))

  model .add(Dense(1))
  model .add(Activation('sigmoid'))

  #optimizer = SGD(lr=0.0001, momentum = 0.9, nesterov=True)
  #optimizer = Adagrad(lr=0.0001)
  #optimizer = Adadelta(rho=0.95)
  optimizer = Adam(lr = 0.0001, beta_1 = 0.9, beta_2 = 0.999)
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  
  return model

def build_model2():
  rmodel = Sequential()
  rmodel.add(Dense(64, input_dim=len_in))
  rmodel.add(BatchNormalization())
  rmodel.add(Activation('relu'))  
  
  rmodel.add(Dense(32))
  rmodel.add(BatchNormalization())
  rmodel.add(Activation('relu'))  
  
  rmodel.add(Dense(16))
  rmodel.add(BatchNormalization())
  rmodel.add(Activation('relu'))  
  
  rmodel.add(Dense(8))
  rmodel.add(BatchNormalization())
  rmodel.add(Activation('relu'))  
  
  rmodel.add(Dense(4))
  rmodel.add(BatchNormalization())
  rmodel.add(Activation('relu')) 
  
  rmodel.add(Dense(1))
  rmodel.add(Activation('sigmoid'))
  
  
  lmodel = Sequential()
  lmodel.add(Dense(64, input_dim=len_in))
  lmodel.add(BatchNormalization())
  lmodel.add(Activation('relu'))  
  
  lmodel.add(Dense(32))
  lmodel.add(BatchNormalization())
  lmodel.add(Activation('relu'))  
  
  lmodel.add(Dense(16))
  lmodel.add(BatchNormalization())
  lmodel.add(Activation('relu'))  
  
  lmodel.add(Dense(8))
  lmodel.add(BatchNormalization())
  lmodel.add(Activation('relu'))  
  
  lmodel.add(Dense(4))
  lmodel.add(BatchNormalization())
  lmodel.add(Activation('relu')) 
  
  lmodel.add(Dense(1))
  lmodel.add(Activation('sigmoid'))
  
  merge = Sequential()
  
  merge = Add([lmodel, rmodel])
  dense1  = Dense(16, activation='relu')(merge)
  dense2  = Dense(8,  activation='relu')(dense1)
  dense2a = BatchNormalization()(dense2)
  dense3  = Dense(8,  activation='relu')(dense2a)
  
  output  = Dense(1, activation='sigmoid')(dense3)
  
  model = Model(inputs=[lmodel, rmodel], outputs=out)
  
  optimizer = Adam(lr = 0.0001, beta_1 = 0.9, beta_2 = 0.999)
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  
  
  return model
# Make models as ensemble
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100)

ensemble_size = 1
models = []
for i in range(ensemble_size):
    models.append(build_model())
    
    
    

In [265]:
# training 시작!!
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor = 'val_loss', patience=100, verbose =1) 
#patience: 오차를 보기 위해서 과거 몇 에폭까지 거슬러 올라가는가

epochs = 1000
batch_size = 100
histories = []  ## for loss curve
print('Training 시작!!')
for i in range(ensemble_size):
    print ('model = ', i)
    hist = models[i].fit(X_train, Y_train, validation_data = (X_valid, Y_valid),
                         epochs=epochs, batch_size=batch_size, callbacks = [early_stopping])
    histories.append(hist)
#model.fit(X_train2, Y_train2, epochs=epochs, batch_size=batch_size)
print('Training 끝!!')

Training 시작!!
model =  0
Train on 712 samples, validate on 179 samples
Epoch 1/1000
712/712 [==============================] - 48s 67ms/step - loss: 0.8707 - acc: 0.5169 - val_loss: 0.7460 - val_acc: 0.5810
Epoch 2/1000
712/712 [==============================] - 0s 207us/step - loss: 0.8163 - acc: 0.5506 - val_loss: 0.7185 - val_acc: 0.6145
Epoch 3/1000
712/712 [==============================] - 0s 218us/step - loss: 0.7572 - acc: 0.5969 - val_loss: 0.6855 - val_acc: 0.6760
Epoch 4/1000
712/712 [==============================] - 0s 205us/step - loss: 0.7336 - acc: 0.6376 - val_loss: 0.6710 - val_acc: 0.6983
Epoch 5/1000
712/712 [==============================] - 0s 203us/step - loss: 0.7165 - acc: 0.6320 - val_loss: 0.6501 - val_acc: 0.7039
Epoch 6/1000
712/712 [==============================] - 0s 202us/step - loss: 0.7050 - acc: 0.6531 - val_loss: 0.6343 - val_acc: 0.6927
Epoch 7/1000
712/712 [==============================] - 0s 199us/step - loss: 0.6698 - acc: 0.6756 - val_loss: 0.

In [274]:
random_forest.fit(rf_X_train, rf_Y_train)
predicted = random_forest.predict(X_test)
rf_sub_data=np.concatenate((p_id,result),axis=1)
rf_sub_data=pd.DataFrame(rf_sub_data,columns=['PassengerId','Survived'])
print(sub_data.head())
rf_sub_data.to_csv('/content/gdrive/My Drive/submission_rf2.csv',index=False)

   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [266]:
#Y_test = model.predict_classes(X_test)
predictions = []                                       # For ensemble averaging   
for i in range(ensemble_size):
    temp = models[i].predict_classes(X_test)
    predictions.append(temp)
result = sum(predictions)
print(result[0:5])
np.place(result, result<=(ensemble_size/2), 0)
np.place(result, result>(ensemble_size/2), 1)
print(result[0:5])

[[0]
 [0]
 [0]
 [0]
 [1]]
[[0]
 [0]
 [0]
 [0]
 [1]]


In [267]:
# 제출 파일 만들기
#Y_test = np.greater(Y_test,0.5).astype(int)
sub_data=np.concatenate((p_id,result),axis=1)
sub_data=pd.DataFrame(sub_data,columns=['PassengerId','Survived'])
print(sub_data.head())
sub_data.to_csv('/content/gdrive/My Drive/submission3.csv',index=False)

   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1


In [229]:
grouped = data_train.groupby(['Sex','Pclass', 'Initial'])
grouped.Age.median()
#data_train.Age = grouped.Age.apply(lambda x: x.fillna(x.median()))

Sex  Pclass  Initial
0    1       0           4.0
             2          36.0
             4          56.0
     2       0           1.0
             2          32.0
             4          46.5
     3       0           5.0
             2          32.0
1    1       1          29.0
             2          49.0
             3          38.5
     2       1          24.0
             3          32.0
     3       1          22.0
             3          36.0
Name: Age, dtype: float64